In [3]:
import warnings
warnings.filterwarnings("ignore")

import yfinance as yf
import pandas as pd
import vectorbt as vbt
import pytz
import numpy as np

# ==============================
# CONFIG
# ==============================
symbol = "^NSEI"
interval = "1m"
period = "7d"

initial_sl_pct = 0.001     # 0.1%
trailing_sl_pct = 0.0001  # 0.03%

timezone = pytz.timezone("Asia/Kolkata")

# # ==============================
# # DOWNLOAD DATA
# # ==============================
# data = yf.download(
#     tickers=symbol,
#     interval=interval,
#     period=period,
#     progress=False
# )

# data.index = data.index.tz_convert(timezone)
# data = data.between_time("09:15", "15:30")

# close = data["Close"].squeeze()

# ==============================
# LOAD DATA FROM CSV
# ==============================
data = pd.read_csv(
    "NIFTY 50_minute.csv",
    parse_dates=["date"]
)

# Set datetime index
data.set_index("date", inplace=True)

# Localize timezone (CSV is usually local exchange time)
data.index = data.index.tz_localize("Asia/Kolkata")

# Ensure proper column naming
data.columns = [c.capitalize() for c in data.columns]

# Filter market hours
data = data.between_time("09:15", "15:30")

# Close price series
close = data["Close"].squeeze()




In [4]:
print(close.index.min(), close.index.max())
print(close.head())
print(close.tail())

2015-01-09 09:15:00+05:30 2026-01-22 12:45:00+05:30
date
2015-01-09 09:15:00+05:30    8292.10
2015-01-09 09:16:00+05:30    8288.15
2015-01-09 09:17:00+05:30    8293.90
2015-01-09 09:18:00+05:30    8300.65
2015-01-09 09:19:00+05:30    8301.20
Name: Close, dtype: float64
date
2026-01-22 12:41:00+05:30    25226.55
2026-01-22 12:42:00+05:30    25213.20
2026-01-22 12:43:00+05:30    25232.30
2026-01-22 12:44:00+05:30    25244.95
2026-01-22 12:45:00+05:30    25246.05
Name: Close, dtype: float64


In [5]:
# ==============================
# INDICATORS
# ==============================
ema7 = vbt.MA.run(close, window=7, ewm=True).ma
ema21 = vbt.MA.run(close, window=21, ewm=True).ma

# ==============================
# ENTRY SIGNAL
# ==============================
entries = ema7.vbt.crossed_above(ema21)

# ==============================
# CREATE TRADE ID (VERY IMPORTANT)
# ==============================
trade_id = entries.cumsum()
trade_id = trade_id.replace(0, np.nan)

# ==============================
# HIGHEST PRICE SINCE ENTRY
# ==============================
highest_price = close.groupby(trade_id).cummax()

# ==============================
# INITIAL STOP LOSS (0.1%)
# ==============================
initial_sl_price = close * (1 - initial_sl_pct)

# ==============================
# TRAILING STOP LOSS (0.03%)
# ==============================
trailing_sl_price = highest_price * (1 - trailing_sl_pct)

# ==============================
# FINAL STOP LOSS (NEVER MOVES DOWN)
# ==============================
final_sl = pd.concat(
    [initial_sl_price, trailing_sl_price],
    axis=1
).max(axis=1)

# ==============================
# STOP LOSS EXIT
# ==============================
sl_exit = close < final_sl

# ==============================
# TIME EXIT (3:15 PM)
# ==============================
time_exit = close.index.time >= pd.to_datetime("15:15").time()

# ==============================
# COMBINED EXITS
# ==============================
exits = sl_exit | time_exit

# ==============================
# PORTFOLIO
# ==============================
pf = vbt.Portfolio.from_signals(
    close=close,
    entries=entries,
    exits=exits,
    direction="longonly",
    freq="1T",
    init_cash=100000
)

# ==============================
# RESULTS
# ==============================
# print(pf.stats())
# pf.plot().show()

In [6]:

# # ==============================
# # RESULTS
# # ==============================
print(pf.stats())
# pf.plot().show()

Start                         2015-01-09 09:15:00+05:30
End                           2026-01-22 12:45:00+05:30
Period                                708 days 07:40:00
Start Value                                    100000.0
End Value                                 151760.460572
Total Return [%]                              51.760461
Benchmark Return [%]                         204.459063
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               7.414706
Max Drawdown Duration                 137 days 08:51:00
Total Trades                                      25023
Total Closed Trades                               25023
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  37.029932
Best Trade [%]                                  1.22351
Worst Trade [%]                               -0

In [8]:
trades_df = pf.trades.records_readable.copy()

for col in trades_df.columns:
    if pd.api.types.is_datetime64tz_dtype(trades_df[col]):
        trades_df[col] = trades_df[col].dt.tz_localize(None)

# trades_df.to_excel("nifty_ema_trades_without_fees_with_SL.xlsx", index=False)
trades_df.to_excel("Testing.xlsx", index=False)